In [1]:
import sys
sys.path.append("..")
from flame_analysis_model import *
from mpi4py import MPI

from sklearn.preprocessing import StandardScaler
import pandas as pd
import sympy
from pysr import PySRRegressor

C:\Users\alexp\Documents\Bachelor Thesis\Code\.venv\Lib\site-packages\juliacall\__init__.py:61: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:

comm = MPI.COMM_WORLD
rank = comm.rank

# Scalar names
scalars = [
    'H2', 'O2', 'H2O', 'H', 'O', 'OH', 'HO2', 'H2O2', 'N2', 'heat_release',
    'curvature', 'stretch_rate', 'disp_speed_density_weighted',
    'abs_flame_prop_vel_normal', 'flow_velocity_normal',
    'flow_velocity_tangential', 'tangential_strain_rate',
    'normal_strain_rate', 'total_strain_rate', 'density_ratio_sigma',
    'vorticity'
]
folder_name = "old_phi0.40_h100"
# Geometry / reference file (constant)
gname = f"./data/{folder_name}/po_premix0.f00001"

# List to collect flame-front slices from all timesteps
front_list = []
timestep_start = 122
timestep_end = 123
var_name = "disp_speed_density_weighted"  #"disp_speed_density_weighted" #"T"
features =[ "curvature", "stretch_rate","total_strain_rate" ]

for timestep in range(timestep_start, timestep_end):
    # Adjust zero padding to match your actual filenames
    print(f"Time step is now {timestep}")
    fname = f"./data/{folder_name}/po_premix0.f{timestep:05d}"

    ds = SEMDataset(
        fname,
        comm=comm,
        gname=gname,
        scalar_names=scalars
    )

    df_full = ds.create_dataframe(
        compute_vel_jacobian=False,
        compute_vel_hessian=False,
        compute_reaction_rates=False,
        cantera_inputs=["BurkeH2.yaml", None, 0.5, 300, 5e05]
    )
    print(f"Timestep {timestep}: points before filtering = {len(df_full)}")
    Tmin = df_full["T"].min()
    Tmax = df_full["T"].max()
    df_full["Tnorm"] = (df_full["T"] - Tmin) / (Tmax - Tmin)
    df_front = df_full[(df_full["Tnorm"] > 0.3) & (df_full["Tnorm"] < 0.8)]
    print(f"Timestep {timestep}: points after filtering = {len(df_front)}")
    # Keep only what we need for DMD
    df = df_front[[var_name] + features].copy()
    df.loc[:, "timestep"] = timestep

    front_list.append(df)
    del df_full

# Concatenate all timesteps into a single DataFrame
front_all = pd.concat(front_list, ignore_index=True)
del front_list

print(front_all.shape)
print(front_all.head())
print(front_all['timestep'].unique())



Time step is now 122
2025-11-23 10:09:27,406 - Mesh - INFO - Initializing empty Mesh object.
2025-11-23 10:09:27,407 - Field - INFO - Initializing empty Field object
2025-11-23 10:09:27,409 - pynekread - INFO - Reading file: ./data/old_phi0.40_h100/po_premix0.f00001
2025-11-23 10:09:27,435 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2025-11-23 10:09:27,436 - Mesh - INFO - Initializing common attributes.
2025-11-23 10:09:27,436 - Mesh - INFO - Getting vertices
2025-11-23 10:09:27,442 - Mesh - INFO - Getting edge centers
2025-11-23 10:09:27,504 - Mesh - INFO - Facet centers not available for 2D
2025-11-23 10:09:27,505 - Mesh - INFO - Mesh object initialized.
2025-11-23 10:09:27,506 - Mesh - INFO - Mesh data is of type: float64
2025-11-23 10:09:27,506 - Mesh - INFO - Elapsed time: 0.07115179998800159s
2025-11-23 10:09:27,506 - pynekread - INFO - Reading field data
2025-11-23 10:09:27,507 - pynekread - INFO - Overwriting fld object
2025-11-23 10:09:27,863 - pynekread - IN

In [3]:
#Prepare data for PySR
X_df = front_all[features].copy()
y_array = front_all[var_name].to_numpy()
# PySR expects numpy or pandas; DataFrame is convenient
print("X_df shape:", X_df.shape)
print("y_array shape:", y_array.shape)

from pysr import PySRRegressor

default_pysr_params = dict(
    populations=30,
    model_selection="best",
)

model = PySRRegressor(
    niterations=50,
    batching= True,
    batch_size= 256,
    binary_operators=["+", "*", "/"],
    unary_operators=["cos", "exp", "sin"]
)

# Fit model
model.fit(X_df, y_array)

print(model)

C:\Users\alexp\Documents\Bachelor Thesis\Code\.venv\Lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...


X_df shape: (57007, 3)
y_array shape: (57007,)


[ Info: Started!



Expressions evaluated per second: 6.250e+03
Progress: 773 / 1550 total iterations (49.871%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.627e+00  0.000e+00  y = 0.8451
3           1.159e+00  1.696e-01  y = -0.01228 * stretch_rate
5           5.743e-01  3.510e-01  y = (stretch_rate * -0.01228) + 0.75625
7           5.426e-01  2.840e-02  y = ((stretch_rate * -0.01228) + 0.71323) + total_strain_r...
                                      ate
9           4.936e-01  4.728e-02  y = ((curvature * -0.15833) + (stretch_rate * -0.010171)) ...
                                      + 0.70096
11          4.914e-01  2.298e-03  y = ((curvature * -0.15833) + ((stretch_rate + curvature) ...
                                      * -0.010171)) + 0.70096
12          4.310e-01  1.311e-01  y = (cos

[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.627e+00  0.000e+00  y = 0.8451
2           1.627e+00  -0.000e+00  y = cos(-0.56375)
3           1.159e+00  3.392e-01  y = -0.01228 * stretch_rate
4           1.044e+00  1.047e-01  y = exp(-0.10269 * curvature)
5           5.742e-01  5.976e-01  y = (stretch_rate + -62.355) * -0.011988
7           5.419e-01  2.896e-02  y = (total_strain_rate + 0.71122) + (stretch_rate * -0.011...
                                      993)
9           4.874e-01  5.299e-02  y = ((stretch_rate * -0.0079597) + (curvature * -0.27929))...
                                       + 0.66956
11          4.683e-01  1.996e-02  y = (curvature * -0.27929) + ((-0.0079597 * stretch_rate) ...
                                      + cos(cos(stretch_rate)))
12          4.298e-01  8.583e-02  y = ((cos(stretch_rate * 0.056921) * -1.1648) + (stretch_r...
                  

In [4]:
model.sympy()

stretch_rate*(-0.011037342) + cos(stretch_rate*0.056921165)*(-1.164798) + 1.8684176